In [ ]:
# Задание-1:
# Написать программу, выполняющую операции (сложение и вычитание) с простыми дробями.
# Дроби вводятся и выводятся в формате:
# n x/y ,где n - целая часть, x - числитель, у - знаменатель.
# Дроби могут быть отрицательные и не иметь целой части, или иметь только целую часть.

import re

def get_sign(num):
    return 1 if num >= 0 else -1

def nod_evklid(a, b):
    while a != 0 and b != 0:
        if a > b:
            a = a % b
        else:
            b = b % a
    return a + b

class Fraction:
    numerator = 0
    denominator = 1

    def __init__(self, fract_str=''):
        if len(fract_str) == 0:
            return

        denominator = re.search('\/(\d+)', fract_str)
        if denominator:
            self.denominator = int(denominator.group(1))
        numerator = re.search('([+-]?\d+)\/', fract_str)
        if numerator:
            self.numerator = int(numerator.group(1))
        if '/' not in fract_str:
            integer = int(fract_str)
            self.numerator += abs(integer * self.denominator)
            self.numerator *= get_sign(integer)
        else:
            integer = re.search('([+-]?\d+)[ ]', fract_str)
            if integer:
                integer = int(integer.group(1))
                self.numerator += abs(integer * self.denominator)
                self.numerator *= get_sign(integer)
        self.simplify()

    def __add__(self, other):
        if isinstance(other, self.__class__):
            f_new = Fraction()
            f_new.numerator = self.numerator * other.denominator + self.denominator * other.numerator
            f_new.denominator = self.denominator * other.denominator
            f_new.simplify()
            return f_new
        elif isinstance(other, int):
            return self + Fraction(str(other))

    __radd__ = __add__

    def __mul__(self, other):
        if isinstance(other, self.__class__):
            f_new = Fraction()
            f_new.numerator = self.numerator * other.numerator
            f_new.denominator = self.denominator * other.denominator
            f_new.simplify()
            return f_new
        elif isinstance(other, int):
            return self * Fraction(str(other))

    __rmul__ = __mul__

    def __sub__(self, other):
        if isinstance(other, self.__class__):
            f_new = Fraction()
            f_new.numerator = self.numerator * other.denominator - self.denominator * other.numerator
            f_new.denominator = self.denominator * other.denominator
            f_new.simplify()
            return f_new
        elif isinstance(other, int):
            return self - Fraction(str(other))

    def __truediv__(self, other):
        if isinstance(other, self.__class__):
            f_new = Fraction()
            f_new.numerator = abs(self.numerator) * other.denominator
            f_new.denominator = self.denominator * abs(other.numerator)
            f_new.numerator *= get_sign(self.numerator * other.numerator)
            f_new.simplify()
            return f_new
        elif isinstance(other, int):
            f_new = Fraction()
            f_new.numerator = self.numerator
            f_new.denominator = self.denominator * abs(other)
            f_new.numerator *= get_sign(self.numerator * other)
            f_new.simplify()
            return f_new

    def __str__(self):
        return '{}{}'.format(self.numerator, f'/{self.denominator}' if self.numerator != 0 else '')

    def humanized(self):
        integer = abs(self.numerator) // self.denominator
        integer *= get_sign(self.numerator)
        numerator = abs(self.numerator) % self.denominator if integer != 0 else self.numerator
        denominator = self.denominator
        if numerator == 0 and integer == 0:
            return '0'
        return '{}{}{}{}'.format(f'{integer}' if integer != 0 else '',
                                 ' ' if integer != 0 else '',
                                 numerator if numerator != 0 else '',
                                 f'/{denominator}' if numerator != 0 else '')

    def simplify(self):
        nod = nod_evklid(abs(self.numerator), self.denominator)
        self.numerator = int(self.numerator / nod)
        self.denominator = int(self.denominator / nod)

    def for_denominator(self, denominator):
        f_new = Fraction()
        f_new.denominator = self.denominator * denominator

def exec_operation(f1_str, f2_str, operation_str, trace=False):
    res = Fraction()
    f1 = Fraction(f1_str)
    f2 = Fraction(f2_str)
    if operation_str == '+':
        res = f1 + f2
    if operation_str == '-':
        res = f1 - f2
    if operation_str == '*':
        res = f1 * f2
    if operation_str == '/':
        res = f1 / f2
    if trace:
        print(f1, operation_str, f2, ' = ', res, sep=' ')
    return res

def get_prior_operation(operators):
    if '*' in operators:
        return '*'
    if '/' in operators:
        return '/'
    return operators[0]

def calc_expression(str, trace=False):
    operators_pattern = '[ ]{1}[-+*\/]{1}[ ]{1}'
    fractions = re.split(operators_pattern, str)
    operators = list(map(lambda x: x.strip(), re.findall(operators_pattern, str)))

    if trace:
        print(operators)
        print(fractions)

    if not operators or len(operators) == 0:
        return Fraction(str).humanized()

    next_operation = get_prior_operation(operators)
    next_operation_index = operators.index(next_operation)
    operators.pop(next_operation_index)

    f2_str = fractions.pop(next_operation_index + 1)
    f1_str = fractions.pop(next_operation_index)
    f_new = exec_operation(f1_str, f2_str, next_operation, trace)

    fractions.insert(next_operation_index, f'{f_new}')

    arr_next = []
    for i in range(len(operators)):
        arr_next.append(fractions[i])
        arr_next.append(operators[i])
    arr_next.append(fractions[-1])

    return calc_expression(' '.join(arr_next), trace)


expr_default = '1 - 1/13 / -1/2 + 2/5'
msg = 'Введите выражение содержащее дроби (поддерживаемые операции +,-,*,/)\n' \
      'Или оставьте поле пустым для выражения по-умолчанию\n'
input_str = input(msg)
expr_to_calc = input_str or expr_default
print(expr_to_calc)
to_trase = input('Включить трейсинг? [y/n] ').lower() == 'y'
print(calc_expression(expr_to_calc, to_trase))


In [ ]:
# Test fractions
input_str_list = ['-1', '1', '0', '1/2', '-1/2', '-2/12', '-1 2/5']

print('String methods:')
print('input', '__str__', 'humanized', sep='\t\t', end='\n\n')
for x in input_str_list:
    f = Fraction(x)
    print(x, f, f.humanized(), sep='\t\t')

f_pos_12 = Fraction('1/2')
f_neg_12 = Fraction('-1/2')

print('')
print('Math methods:')
print('__str__', 'add1/2', 'add-1/2', 'sub1/2', 'sub-1/2', sep='\t\t', end='\n\n')
for x in input_str_list:
    f = Fraction(x)
    print(f, f + f_pos_12, f + f_neg_12, f - f_pos_12, f - f_neg_12, sep='\t\t')
    
print('')
print('Math methods p2:')
print('__str__', 'mul1/2', 'mul-1/2', 'div1/2', 'div-1/2', sep='\t\t', end='\n\n')
for x in input_str_list:
    f = Fraction(x)
    print(f, f * f_pos_12, f * f_neg_12, f / f_pos_12, f / f_neg_12, sep='\t\t')
